In [8]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import yfinance as yf

import numpy as np
import pandas as pd


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor

import tensorflow as tf
from keras._tf_keras.keras.models import Model
from keras._tf_keras.keras.layers import Dense, Input, Dropout, BatchNormalization, LeakyReLU
from keras._tf_keras.keras.regularizers import l2

In [9]:
crypto = ['BTC-USD','BNB-USD', 'ETH-USD','SOL-USD']

In [3]:
c = crypto[1]
df0 = yf.download(c, period='max')
df0.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-09-25,606.523071,607.762939,584.858826,587.352539,587.352539,1744393946
2024-09-26,587.351379,608.854187,581.228333,596.776917,596.776917,2081857126
2024-09-27,596.776917,614.473511,596.500977,607.867004,607.867004,2144197609
2024-09-28,607.867004,617.332397,595.207764,601.567200,601.567200,1858159482
2024-09-29,601.451599,602.541443,593.228027,597.609314,597.609314,1599390976


In [10]:
def build_model(units=128, optimizer='adam', activation='relu', dropout_rate=0.3, regularization_rate=0.01):
    i = Input(shape=(Tx,))
    
    # First dense layer with LeakyReLU and batch normalization
    x = Dense(units, kernel_regularizer=l2(regularization_rate))(i)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Second dense layer with LeakyReLU and batch normalization
    x = Dense(units * 2, kernel_regularizer=l2(regularization_rate))(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Third dense layer with ReLU and batch normalization
    x = Dense(units * 2, activation=activation, kernel_regularizer=l2(regularization_rate))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate * 0.5)(x)  # Reduced dropout for later layers
    
    # Output layer
    output = Dense(Ty)(x)
    
    model = Model(inputs=i, outputs=output)
    model.compile(loss='mape', optimizer=optimizer)
    
    return model

In [11]:
param_grid = {
    'model__units': [64, 128],  # Increase the range of units
    'model__activation': ['relu', 'tanh', 'elu'],  # Add LeakyReLU and ELU
    'model__optimizer': ['adam', 'rmsprop', 'nadam'],  # Include Nadam and AdamW
    'model__dropout_rate': [0.1, 0.2],  # Wider range of dropout rates
    'model__regularization_rate': [ 0.001, 0.01],  # Explore finer regularization rates
    'epochs': [50, 100],  # Increase maximum epochs for deeper models
}


In [12]:
model = KerasRegressor(model=build_model, verbose=2)

In [13]:
m = 0
future_day_array = []
for c in crypto:
    df0 = yf.download(c, period='max')
    df0 = df0[-1500:]
    df = df0[['Close']].copy()
    df['LogClose'] = np.log(df['Close'])
    df['DiffLogClose'] = df['LogClose'].diff()
    train = df
    
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train[['DiffLogClose']])
    
    train_idx = df.index <= train.index[-1]
    df.loc[train_idx, 'ScaledLogReturn'] = train_scaled.flatten()
    
    series = df['ScaledLogReturn'].dropna().to_numpy()

    Tx = 50
    Ty = 10
    X = []
    Y = []
    for t in range(len(series) - Tx - Ty + 1):
      x = series[t:t+Tx]
      X.append(x)
      y = series[t+Tx:t+Tx+Ty]
      Y.append(y)
    X = np.array(X).reshape(-1, Tx)
    Y = np.array(Y).reshape(-1, Ty)
    
    Xtrain_m, Ytrain_m = X, Y
    
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2, verbose=2)
    grid_result = grid.fit(Xtrain_m, Ytrain_m)
    best_model = grid_result.best_estimator_
        
    future_day_scaled_log_diff = best_model.predict(Xtrain_m[-10:])
    future_day_scaled_log_diff = future_day_scaled_log_diff[:,0]
    future_day_log_diff = scaler.inverse_transform(future_day_scaled_log_diff.reshape(-1,1)).flatten()
    last_train = train.iloc[-1]['LogClose']
    future_day_log = last_train + np.cumsum(future_day_log_diff)
    future_day = np.exp(future_day_log)
    print('For crypto:',c,'today was',df['Close'].iloc[-1],'and tomorrow will be :',future_day[Ty-1])
    print('So difference for:',c,'is:',future_day[Ty-1] - df['Close'].iloc[-1],'which is per dollar:',round((future_day[Ty-1] - df['Close'].iloc[-1])/future_day[Ty-1],5))

[*********************100%***********************]  1 of 1 completed


Fitting 2 folds for each of 144 candidates, totalling 288 fits


C:\Users\Korisnik\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Epoch 1/100
45/45 - 2s - 37ms/step - loss: 837.7582
Epoch 2/100
45/45 - 0s - 2ms/step - loss: 770.6459
Epoch 3/100
45/45 - 0s - 2ms/step - loss: 662.1416
Epoch 4/100
45/45 - 0s - 2ms/step - loss: 649.8273
Epoch 5/100
45/45 - 0s - 2ms/step - loss: 599.6901
Epoch 6/100
45/45 - 0s - 2ms/step - loss: 522.7653
Epoch 7/100
45/45 - 0s - 2ms/step - loss: 563.0628
Epoch 8/100
45/45 - 0s - 2ms/step - loss: 503.8659
Epoch 9/100
45/45 - 0s - 2ms/step - loss: 462.2401
Epoch 10/100
45/45 - 0s - 2ms/step - loss: 439.0882
Epoch 11/100
45/45 - 0s - 2ms/step - loss: 447.3350
Epoch 12/100
45/45 - 0s - 2ms/step - loss: 410.9137
Epoch 13/100
45/45 - 0s - 2ms/step - loss: 415.4202
Epoch 14/100
45/45 - 0s - 2ms/step - loss: 385.6503
Epoch 15/100
45/45 - 0s - 2ms/step - loss: 362.3763
Epoch 16/100
45/45 - 0s - 2ms/step - loss: 329.0608
Epoch 17/100
45/45 - 0s - 2ms/step - loss: 351.4658
Epoch 18/100
45/45 - 0s - 2ms/step - loss: 314.6415
Epoch 19/100
45/45 - 0s - 2ms/step - loss: 322.1246
Epoch 20/100
45/45 -

[*********************100%***********************]  1 of 1 completed


...

Fitting 2 folds for each of 144 candidates, totalling 288 fits


C:\Users\Korisnik\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/100


C:\Users\Korisnik\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


45/45 - 2s - 55ms/step - loss: 747.8218
Epoch 2/100
45/45 - 0s - 3ms/step - loss: 652.5847
Epoch 3/100
45/45 - 0s - 2ms/step - loss: 594.6823
Epoch 4/100
45/45 - 0s - 2ms/step - loss: 535.3329
Epoch 5/100
45/45 - 0s - 2ms/step - loss: 506.5260
Epoch 6/100
45/45 - 0s - 2ms/step - loss: 445.1516
Epoch 7/100
45/45 - 0s - 3ms/step - loss: 447.8998
Epoch 8/100
45/45 - 0s - 3ms/step - loss: 415.8036
Epoch 9/100
45/45 - 0s - 2ms/step - loss: 422.8457
Epoch 10/100
45/45 - 0s - 2ms/step - loss: 396.9376
Epoch 11/100
45/45 - 0s - 2ms/step - loss: 395.2333
Epoch 12/100
45/45 - 0s - 3ms/step - loss: 364.6440
Epoch 13/100
45/45 - 0s - 2ms/step - loss: 349.6243
Epoch 14/100
45/45 - 0s - 3ms/step - loss: 373.7072
Epoch 15/100
45/45 - 0s - 2ms/step - loss: 338.3277
Epoch 16/100
45/45 - 0s - 2ms/step - loss: 335.3015
Epoch 17/100
45/45 - 0s - 2ms/step - loss: 325.7258
Epoch 18/100
45/45 - 0s - 2ms/step - loss: 309.8746
Epoch 19/100
45/45 - 0s - 2ms/step - loss: 297.9003
Epoch 20/100
45/45 - 0s - 2ms/st